<a href="https://colab.research.google.com/github/LeNovice/PSW/blob/main/IDS_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Group No

## Group Member Names:
1. 
2.
3.



# 1. Business Understanding
 
Students are expected to identify a classification problem of your choice. You have to detail the Business Understanding part of your problem under this heading which basically addresses the following questions.
 
   1. What is the problem that you are trying to solve?
   2. What data do you need to answer the above problem?
   3. What are the different sources of data?    
   4. What kind of analytics task are you performing?



In [106]:
import pandas as pd
from sklearn.impute import SimpleImputer

# 2. Data Acquisition
 
For the problem identified by you, students have to find the data source themselves which should be a website which has the required data in it. You have to write Python crawler code to scrape data from the respective website rather than downloading ready-made dataset as such from sources like Kaggle etc. 

(Data downloaded from website like Kaggle will be awarded negative marks.)

## 2.1 Code for scraping data from website



In [107]:
##---------Type the code below this line------------------##


## 2.2 Code for converting the above scraped data into a dataframe

In [108]:
##---------Type the code below this line------------------##
raw_DF = pd.read_csv('https://raw.githubusercontent.com/ParvN/Chronic-Kidney-Disease-Classification/master/ckd.csv',sep=',', na_values=['?'])

## 2.3 Confirm the data has been correctly by displaying the first 5 and last 5 records.

In [109]:
##---------Type the code below this line------------------##
print("Top 5")
display(raw_DF.head())
print("Bottom 5")
display(raw_DF.tail())

Top 5


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


Bottom 5


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,49.0,0.5,150.0,4.9,15.7,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,31.0,1.2,141.0,3.5,16.5,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,26.0,0.6,137.0,4.4,15.8,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,50.0,1.0,135.0,4.9,14.2,51.0,7200.0,5.9,no,no,no,good,no,no,notckd
399,58.0,80.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,131.0,18.0,1.1,141.0,3.5,15.8,53.0,6800.0,6.1,no,no,no,good,no,no,notckd


## 2.4 Display the column headings, statistical information, description and statistical summary of the data.

In [110]:
##---------Type the code below this line------------------##
DF_columns = raw_DF.columns.values
print(DF_columns)
display(raw_DF.describe(percentiles=None))
display(raw_DF.dtypes)
print("\nShape of DataFrame" + str(raw_DF.shape))

['age' 'bp' 'sg' 'al' 'su' 'rbc' 'pc' 'pcc' 'ba' 'bgr' 'bu' 'sc' 'sod'
 'pot' 'hemo' 'pcv' 'wbcc' 'rbcc' 'htn' 'dm' 'cad' 'appet' 'pe' 'ane'
 'class']


,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
count,391.000000,388.000000,353.000000,354.000000,351.000000,356.000000,381.000000,383.000000,313.000000,312.000000,348.000000,329.000000,294.000000,269.000000
mean,51.483376,76.469072,1.017408,1.016949,0.450142,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437,38.884498,8406.122449,4.707435
std,17.169714,13.683637,0.005717,1.352679,1.099191,79.281714,50.503006,5.741126,10.408752,3.193904,2.912587,8.990105,2944.474190,1.025323
min,2.000000,50.000000,1.005000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000,9.000000,2200.000000,2.100000
25%,42.000000,70.000000,1.010000,0.000000,0.000000,99.000000,27.000000,0.900000,135.000000,3.800000,10.300000,32.000000,6500.000000,3.900000
50%,55.000000,80.000000,1.020000,0.000000,0.000000,121.000000,42.000000,1.300000,138.000000,4.400000,12.650000,40.000000,8000.000000,4.800000
75%,64.500000,80.000000,1.020000,2.000000,0.000000,163.000000,66.000000,2.800000,142.000000,4.900000,15.000000,45.000000,9800.000000,5.400000
max,90.000000,180.000000,1.025000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000,54.000000,26400.000000,8.000000


age      float64
bp       float64
sg       float64
al       float64
su       float64
rbc       object
pc        object
pcc       object
ba        object
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wbcc     float64
rbcc     float64
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
class     object
dtype: object


Shape of DataFrame(400, 25)


## 2.5 Write your observations from the above. 
1. Size of the dataset

As displayed in above code snippet, Size of dataset is 400 rows and 25 columns including the target variable
2. What type of data attributes are there?
3. Is there any null data that has to be cleaned?


# 3. Data Preparation

## 3.1 Display how many unique values are present in each attribute


In [111]:
##---------Type the code below this line------------------##
for i in DF_columns:
    print("Number of Unique values for attribute " + i + " = " , raw_DF[i].unique().shape[0])

Number of Unique values for attribute age =  77
Number of Unique values for attribute bp =  11
Number of Unique values for attribute sg =  6
Number of Unique values for attribute al =  7
Number of Unique values for attribute su =  7
Number of Unique values for attribute rbc =  3
Number of Unique values for attribute pc =  3
Number of Unique values for attribute pcc =  3
Number of Unique values for attribute ba =  3
Number of Unique values for attribute bgr =  147
Number of Unique values for attribute bu =  119
Number of Unique values for attribute sc =  85
Number of Unique values for attribute sod =  35
Number of Unique values for attribute pot =  41
Number of Unique values for attribute hemo =  116
Number of Unique values for attribute pcv =  43
Number of Unique values for attribute wbcc =  90
Number of Unique values for attribute rbcc =  46
Number of Unique values for attribute htn =  3
Number of Unique values for attribute dm =  3
Number of Unique values for attribute cad =  3
Numbe

## 3.2 Check for the presence of duplicate data, identify the attributes with duplicate data, report the attributes. Mention the method adopted to remove duplicate data if present. Report the results again.

In [112]:
##---------Type the code below this line------------------##
print("Number of duplicates =", raw_DF.duplicated().sum())
display()
print("DataFrame shape before dropping duplicates", raw_DF.shape)
raw_DF.drop_duplicates(subset=None, keep='first', inplace=True)
print("DataFrame shape after dropping duplicates", raw_DF.shape)

Number of duplicates = 0
DataFrame shape before dropping duplicates (400, 25)
DataFrame shape after dropping duplicates (400, 25)


## 3.3 Show whether there are any missing values in each attribute. Report the same.


In [113]:
##---------Type the code below this line------------------##
missing_DF = raw_DF.isnull()
for attr in missing_DF.columns:
    print("Number of missing values for attribute " + attr +" =", missing_DF[attr].sum())

Number of missing values for attribute age = 9
Number of missing values for attribute bp = 12
Number of missing values for attribute sg = 47
Number of missing values for attribute al = 46
Number of missing values for attribute su = 49
Number of missing values for attribute rbc = 152
Number of missing values for attribute pc = 65
Number of missing values for attribute pcc = 4
Number of missing values for attribute ba = 4
Number of missing values for attribute bgr = 44
Number of missing values for attribute bu = 19
Number of missing values for attribute sc = 17
Number of missing values for attribute sod = 87
Number of missing values for attribute pot = 88
Number of missing values for attribute hemo = 52
Number of missing values for attribute pcv = 71
Number of missing values for attribute wbcc = 106
Number of missing values for attribute rbcc = 131
Number of missing values for attribute htn = 2
Number of missing values for attribute dm = 2
Number of missing values for attribute cad = 2
N

## 3.4 Clean the missing data using any imputation technique, mention the method used and again report the change after cleaning the data.

In [114]:
##---------Type the code below this line------------------##
numerical_columns = ['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wbcc','rbcc']
nominal_columns = ['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane','class']
imputer_nominal = SimpleImputer(strategy="most_frequent")
imputer_numeric = SimpleImputer(strategy='mean')
raw_DF[nominal_columns] = imputer_nominal.fit_transform(raw_DF[nominal_columns])
raw_DF[numerical_columns] = imputer_numeric.fit_transform(raw_DF[numerical_columns])

missing_DF = raw_DF.isnull()
for attr in missing_DF.columns:
    print("Number of missing values After cleaning the data for attribute " + attr +" =", missing_DF[attr].sum())

Number of missing values After cleaning the data for attribute age = 0
Number of missing values After cleaning the data for attribute bp = 0
Number of missing values After cleaning the data for attribute sg = 0
Number of missing values After cleaning the data for attribute al = 0
Number of missing values After cleaning the data for attribute su = 0
Number of missing values After cleaning the data for attribute rbc = 0
Number of missing values After cleaning the data for attribute pc = 0
Number of missing values After cleaning the data for attribute pcc = 0
Number of missing values After cleaning the data for attribute ba = 0
Number of missing values After cleaning the data for attribute bgr = 0
Number of missing values After cleaning the data for attribute bu = 0
Number of missing values After cleaning the data for attribute sc = 0
Number of missing values After cleaning the data for attribute sod = 0
Number of missing values After cleaning the data for attribute pot = 0
Number of miss

## 3.5 Check if all the attributes are following the same format and are consistent. If not, report all such attributes and what inconsistencies are present.

In [ ]:
##---------Type the code below this line------------------##

## 3.6 Correct the data if there are inconsistencies from 3.5. Report or print the data after correction.

In [ ]:
##---------Type the code below this line------------------##

## 3.7 Identify the target variables.


In [ ]:
##---------Type the code below this line------------------##

## 3.8 Separate the data front the target such that the dataset is in the form of (X,y) or (Features, Label)


In [ ]:
##---------Type the code below this line------------------##

## 3.9 Discretize the target variable or perform one-hot encoding on the target or any other as and if required.


In [ ]:
##---------Type the code below this line------------------##

# 4. Data Exploration using various plots

## 4.1 Scatter plot of each attribute with the target.

In [ ]:
##---------Type the code below this line------------------##

## 4.2 Pair plot of each attribute to identify the linear relationships among the attributes.


In [ ]:
##---------Type the code below this line------------------##

## 4.3 Regression plots to identify the linear relationship between each attribute with the target variable. 


In [ ]:
##---------Type the code below this line------------------##

## 4.4 Can any other plot help to identify the optimal set of attributes that can be used for classification. The plot will be based on linear or nonlinear separations. If there is/are such plots, name them, explain why you think they can be helpful in the task and perform the plot as well.


In [ ]:
##---------Type the code below this line------------------##

# 5. Data Wrangling



## 5.1 Display correlation heatmap of each attribute against the target and report which features are significant.

In [ ]:
##---------Type the code below this line------------------##

## 5.2 Univariate Filters – Identify top 5 significant features by evaluating each feature independently with respect to the target variable by exploring 
1. Mutual Information (Information Gain)
2. Gini index
3. Gain Ratio
4. Chi-Squared test
5. Fisher Score
(From the above 5 you are required to use only any <b>three</b>) 

Write your observations from the results of each method and report the top 5 significant features for each of the above methods. Also plot a graph of significant features for each of them for better visualization.

In [ ]:
##---------Type the code below this line------------------##

## 5.3 Train a “DecisionTreeClassifier” on the entire data and use the classifier to extract the top 5 significant features. Plot graph of significant features for better visualization.

In [ ]:
##---------Type the code below this line------------------##

## 5.4 Using "mlxtend" library perform SequentialFeatureSelector to identify top 5 features.

In [ ]:
##---------Type the code below this line------------------##

## 5.5 Conclude the top 3 significant features with necessary justifications.

--------------Type the answers below this line--------------